In [180]:
import pandas as pd
import numpy as np

In [181]:
df = pd.read_csv("./data/Об'єднана_таблиця.csv", sep='\t')

In [182]:
# list of regions
regions = ['Вінницька область', 'Волинська область','Дніпропетровська область']

### Count candidates by party and council

In [183]:
def get_candidates_stat(df, region, elected = False):
    '''
    Function to get pivot table of number of candidates/elected candidates from parties in city councils.
    '''
    obl = region
    header = 'Кількість кандидатів'
    if elected == False:
        pivot_df = pd.pivot_table(df[df['Область'] == obl], index="Партія", columns="Міська рада", values='Статус', aggfunc=len, fill_value=0)
    elif elected == True:
        pivot_df = pd.pivot_table(df[df['Область'] == obl], index="Партія", columns="Міська рада", values='Статус', aggfunc=lambda x:(x=='обрано').sum(), fill_value=0)
        header = 'Кількість обраних кандидатів'
    pivot_df.columns = pd.MultiIndex.from_tuples(zip([header]*len(pivot_df.columns), pivot_df.columns))
    return pivot_df

In [184]:
# count all candidates
vinn_cand, vol_cand, dnipr_cand = [get_candidates_stat(df, i) for i in regions]

In [185]:
# count elected candidates
vinn_elect, vol_elect, dnipr_elect = [get_candidates_stat(df, i, elected=True) for i in regions]

### Merge all candidates and elected candidates dfs

In [186]:
counted_lst = [[vinn_cand, vinn_elect], [vol_cand, vol_elect], [dnipr_cand, dnipr_elect]]
vinn_joined, vol_joined, dnipr_joined = [i.merge(j, how='left', left_on='Партія', right_on='Партія') for (i, j) in counted_lst]

### Save output

In [187]:
vinn_joined.to_csv('./data/Вінницька_область_дані.csv')
vol_joined.to_csv('./data/Волинська_область_дані.csv')
dnipr_joined.to_csv('./data/Дніпропетровська_область_дані.csv')